In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from collections import Counter,deque
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_openml
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score



In [2]:
individual_household_electric_power_consumption = fetch_ucirepo(id=235) 
  
X = individual_household_electric_power_consumption.data.features 
y = individual_household_electric_power_consumption.data.targets 

print("The shape of the data is: ",X.shape)

The shape of the data is:  (2075259, 9)


/Users/jenishkothari/Northeastern/DS5230/HWK2B/myenv/lib/python3.13/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [3]:
class DBScanCustom:
    def __init__(self,minPts,epsilon):
        self.minPts = minPts
        self.epsilon = epsilon
        self.labels = None
        self.dataset = None

    def preProcessing(self,dataset):
        distMatrix = cdist(dataset,dataset,metric='euclidean')       
        data = []
        for i in range(len(dataset)):
            neighbors = [str(j) for j in range(len(dataset)) if i!=j and distMatrix[i][j] <= self.epsilon]
            data.append([i] + [neighbors] + [len(neighbors)])
        self.dataset = data

    def expand(self,currPt,currClusterId):
        self.labels[currPt] = currClusterId
        queue = deque([currPt])
        visited = set()

        while queue:
            inQueue = queue.popleft()
            inQueueNeighborsCnt = self.dataset[inQueue][2]
            visited.add(int(inQueue))

            if inQueueNeighborsCnt>=self.minPts:
                for n in self.dataset[inQueue][1]:
                    n = int(n)
                    if n in visited:
                        continue
                    if self.labels[n] == -1 and self.dataset[n][2] >= self.minPts:
                        self.labels[n] = currClusterId
                        queue.append(n)
                    
    def runDBSCAN(self,dataset):  
        self.labels = [-1]*len(dataset)
        startClusterId = 0
        self.preProcessing(dataset)
 
        for i in range(len(self.dataset)):
            if self.labels[i] != -1:
                continue
            
            neighbors = self.dataset[i][1]
            if len(neighbors) >= self.minPts:
                startClusterId += 1
                self.expand(i,startClusterId)
            
        return self.labels

In [4]:
X = pd.DataFrame(X)
X = X.iloc[:, 2:]
X = X.apply(pd.to_numeric, errors='coerce')
print("The shape of the data is: ", X.shape)
X = X.fillna(X.mean())

print(X)

The shape of the data is:  (2075259, 7)
         Global_active_power  Global_reactive_power  Voltage  \
0                      4.216                  0.418   234.84   
1                      5.360                  0.436   233.63   
2                      5.374                  0.498   233.29   
3                      5.388                  0.502   233.74   
4                      3.666                  0.528   235.68   
...                      ...                    ...      ...   
2075254                0.946                  0.000   240.43   
2075255                0.944                  0.000   240.00   
2075256                0.938                  0.000   239.82   
2075257                0.934                  0.000   239.70   
2075258                0.932                  0.000   239.55   

         Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
0                    18.4             0.0             1.0            17.0  
1                    23.0             0

In [5]:
subset_size = 20000
indices = np.random.choice(len(X), subset_size, replace=False)
X_subset = X.iloc[indices]
dbscan = DBScanCustom(epsilon=2.5, minPts=13)
labels = dbscan.runDBSCAN(X_subset)

subset_df = pd.DataFrame(X.iloc[indices].copy())
subset_df['cluster'] = labels

In [23]:
subset_df['cluster'].value_counts()

cluster
 1    18532
-1     1101
 2      180
 4      107
 3       60
 5       20
Name: count, dtype: int64

In [24]:
import numpy as np

valid_indices = np.where(np.array(labels) != -1)[0]

X_valid = X_subset.iloc[valid_indices]

labels_valid = np.array(labels)[valid_indices].flatten()

if len(np.unique(labels_valid)) > 1:
    score = silhouette_score(X_valid, labels_valid)
    print(f"Silhouette Score: {score}")
else:
    print("Silhouette Score cannot be computed with less than 2 clusters.")


Silhouette Score: 0.7146337318197404
